In [ ]:
import serial, time, csv, math, pandas as pd, pyvisa, glob, matplotlib.pyplot as plt, os.path, numpy as np, pathlib, datetime;
from csv import reader
import sys

In [ ]:
#plotting code

#find intersection point with horizontal 3dB line
def find_roots(x,y): 
    s = np.abs(np.diff(np.sign(y))).astype(bool)
    return x[:-1][s] + np.diff(x)[s]/(np.abs(y[1:][s]/y[:-1][s])+1) 

homedir = '/Volumes/GoogleDrive/.shortcut-targets-by-id/1OcwVYdCFsNitauuq7YD82J3wd781RFEz/Zhao_Nicole /jupyter_homedir/BPF_results/test/'
os.chdir(homedir)
#make a list of all folders
dirs = [d for d in os.listdir(homedir) if os.path.isdir(os.path.join(homedir, d))]

for d in dirs: 
    os.chdir(homedir + '/' + d) 
    #define plot space
    fig, ax = plt.subplots(1,1, figsize=(12,8))
    print (d)
    #create list to store x & y values pulled from files
    xs = [] #bandpass freq
    ys = [] #vout/vin
    zs = [] #store Vin freq (Hz) values
    colors = ['g','y','c']
    output = [] #output value for text file
    
    #write values to csv file
    fname = str("data_" + d) 
    fpath = (homedir + '/' + d/ fname).with_suffix('.csv')
    with fpath.open(mode='w+') as csvfile:
        logfile = csv.writer(csvfile)
        logfile.writerow(["# SN, Timestamp, f0Nominal_Hz, f0Meas_Hz, BW_3dB, Ratio"])
        
    #import and read file
    file_list = glob.glob(homedir + '/' + d + '/*.csv')
    file_list.sort() 
    for file in file_list:
        data = pd.read_csv(file)
        print(file)
        xs.append(data["BP freq (Hz)"][0])
        ys.append(data["Vout dmm (V)"]/data["Vin (V)"])
        zs.append(data["Vin freq (Hz)"]) 

    #iterate over each center freq
    for ifc in range(len(xs)): 
        xmin = min(zs[ifc]) #establish min & max freq range on xaxis based on data set
        xmax = max(zs[ifc]) 

        #have values interpolated
        f = interpolate.interp1d(zs[ifc],ys[ifc],'cubic')
        x_interp = np.arange(xmin,xmax,0.01) #map to 1/100 hz 
        y_interp = f(x_interp) #map y against x
        
        #plot real data as points, plot interp data as ---
        plt.scatter(zs[ifc],ys[ifc],s=8,c=colors[ifc])
        #plot interp data as line
        plt.plot(x_interp, y_interp, color=colors[ifc], linestyle='dashed',label= str(xs[ifc]) + 'hz',alpha=0.5)
        
        #plot zero
        roots = find_roots(x_interp,10**(-3/20)-y_interp) #find roots at y=-3dB
        ax.axhline(y=10**(-3/20),color='red',ls=":") #plot 3dB line in red
        ax.plot(roots,np.interp(roots, x_interp, y_interp),color="b") #plot 3dB bandwidth in blue 

        for root in roots:
            ax.axvline(x=root,color="red",ls=":") #graph vertical line at each zero

        #find important values 
        print ("np.max(y):"+ str(np.max(y_interp)))
        f0 = x_interp[np.argmax(y_interp)] #xmax; 1 decimal place 
        print(f0)
        BW3dB = float(np.diff(roots)) #find 3dB bandwidth
        ratio = f0/BW3dB #ratio of f0/BW3dB
        plt.text(400,0.004+0.02*ifc,"f_0=" + "{:.2f}".format(f0) + "Hz"  + "\nBW_3dB=" 
               + "{:.2f}".format(BW3dB) + "Hz" + "\nRatio=" + "{:.2f}".format(ratio),fontsize=10, bbox = dict(facecolor = 'blue', alpha = 0.2,edgecolor='blue')) 

        #write data into csv file for each bpf
        output.append([d[3:6],d[7:],xs[ifc],"{:.2f}".format(f0),"{:.2f}".format(BW3dB), 
                       "{:.2f}".format(ratio)])
        print(output)
   
    #store output value to csv file
    
    with open('data_' + d, 'a') as f:
        #f.writelines(output)
        #f.close()
        writer = csv.writer(f)
        writer.writerows(output)
    
    #write to csv file 
    logfile.writerow([d[3:6],d[7:],xs[ifc],"{:.2f}".format(f0),"{:.2f}".format(BW3dB), 
                       "{:.2f}".format(ratio)])

    # Set tick font size
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(16)
    
    ax.set_title("bode_" + d, fontsize=16)
    ax.set_xlabel("Frequency (Hz)", fontsize=16)
    ax.set_ylabel("$V_{out}/V_{in}$", fontsize=16)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim(10,1000)
    ax.set_ylim(10**-2.5,1.3)
    plt.legend(prop={"size":16})
    plt.savefig("bode_" + d + ".png")
    plt.show()
    plt.clf()
